RL policy based on the [SoloParkour: Constrained Reinforcement Learning for Visual Locomotion from Privileged Experience](https://arxiv.org/abs/2409.13678). 

# Flat Ground

## Test In Simulation

In [1]:
from Go2Py.robot.fsm import FSM
from Go2Py.robot.remote import KeyboardRemote
from Go2Py.robot.safety import SafetyHypervisor
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.control.cat import *
import torch

In [2]:
robot = Go2Sim()

In [3]:
remote = KeyboardRemote()
robot.sitDownReset()
safety_hypervisor = SafetyHypervisor(robot)

In [4]:
class CaTController:
    def __init__(self, robot, remote, checkpoint):
        self.remote = remote
        self.robot = robot
        self.policy = Policy(checkpoint)
        self.command_profile = CommandInterface()
        self.agent = CaTAgent(self.command_profile, self.robot)
        self.init()

    def init(self):
        self.obs = self.agent.reset()
        self.policy_info = {}
        self.command_profile.yaw_vel_cmd = 0.0
        self.command_profile.x_vel_cmd = 0.0
        self.command_profile.y_vel_cmd = 0.0

    def update(self, robot, remote):
        action = self.policy(self.obs, self.policy_info)
        self.obs, self.ret, self.done, self.info = self.agent.step(action)

In [5]:
from Go2Py import ASSETS_PATH 
import os
checkpoint_path = os.path.join(ASSETS_PATH, 'checkpoints/SoloParkour/go2_flat.pt')
controller = CaTController(robot, remote, checkpoint_path)
fsm = FSM(robot, remote, safety_hypervisor, user_controller_callback=controller.update)

/home/Go2py/Go2Py/control/cat.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  actor_sd = torch.load(checkpoint_path)


p_gains: [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [6]:
controller.command_profile.x_vel_cmd=-0.3
controller.command_profile.y_vel_cmd=0.3
controller.command_profile.yaw_vel_cmd = 0.0

Pressing `u` on the keyboard will make the robot stand up. This is equivalent to the `L2+A` combo of the Go2 builtin state machine. After the the robot is on its feet, pressing `s` will hand over the control the RL policy. This action is equivalent to the `start` key of the builtin controller. When you want to stop, pressing `u` again will act similarly to the real robot and locks it in standing mode. Finally, pressing `u` again will command the robot to sit down.

In [7]:
fsm.close()

## Test on Real Robot (ToDo)

In [ ]:
from Go2Py.robot.fsm import FSM
from Go2Py.robot.remote import UnitreeRemote
from Go2Py.robot.safety import SafetyHypervisor
from Go2Py.control.walk_these_ways import *

In [ ]:
from Go2Py.robot.interface import GO2Real
import numpy as np
robot = GO2Real(mode='lowlevel')

In [ ]:
remote = UnitreeRemote(robot)
safety_hypervisor = SafetyHypervisor(robot)

In [ ]:
class walkTheseWaysController:
    def __init__(self, robot, remote, checkpoint):
        self.remote = remote
        self.robot = robot
        self.cfg = loadParameters(checkpoint)
        self.policy = Policy(checkpoint)
        self.command_profile = CommandInterface()
        self.agent = WalkTheseWaysAgent(self.cfg, self.command_profile, self.robot)
        self.agent = HistoryWrapper(self.agent)
        self.init()

    def init(self):
        self.obs = self.agent.reset()
        self.policy_info = {}
        self.command_profile.yaw_vel_cmd = 0.0
        self.command_profile.x_vel_cmd = 0.0
        self.command_profile.y_vel_cmd = 0.0
        self.command_profile.stance_width_cmd=0.25
        self.command_profile.footswing_height_cmd=0.08
        self.command_profile.step_frequency_cmd = 3.0
        self.command_profile.bodyHeight = 0.00

    def update(self, robot, remote):
        action = self.policy(self.obs, self.policy_info)
        self.obs, self.ret, self.done, self.info = self.agent.step(action)
        vy = -robot.getRemoteState().lx
        vx = robot.getRemoteState().ly
        omega = -robot.getRemoteState().rx*2.2
        self.command_profile.x_vel_cmd = vx*1.5
        self.command_profile.y_vel_cmd = vy*1.5
        self.command_profile.yaw_vel_cmd = omega

In [ ]:
checkpoint = "../Go2Py/assets/checkpoints/walk_these_ways/"
controller = walkTheseWaysController(robot, remote, checkpoint)
safety_hypervisor = SafetyHypervisor(robot)

In [ ]:
controller.command_profile.pitch_cmd=0.0
controller.command_profile.body_height_cmd=0.0
controller.command_profile.footswing_height_cmd=0.04
controller.command_profile.roll_cmd=0.0
controller.command_profile.stance_width_cmd=0.2
controller.command_profile.x_vel_cmd=-0.2
controller.command_profile.y_vel_cmd=0.01
controller.command_profile.setGaitType("trotting")

In [ ]:
fsm = FSM(robot, remote, safety_hypervisor, user_controller_callback=controller.update)

Pressing `L2+A` to command the robot to stand up. After the the robot is on its feet, pressing `start` will hand over the control the RL policy. When you want to stop, pressing `L2+A` again will act similarly to the factory controller and locks the robot in standing mode. Finally, pressing `L2+A` again will command the robot to sit down.

In [ ]:
fsm.close()